# Cargar Imports y libreria de happytransformer

In [ ]:
pip install happytransformer

In [ ]:
from happytransformer import HappyGeneration
from happytransformer import GENSettings
from happytransformer import HappyQuestionAnswering
import pandas as pd

# Carga del modelo GPT-NEO
Por ahora estuve cargando el modelo de 125M ya que el modelo de NEOX pide mucho mas recursos y puede que el colab te termine usando todos los recursos, Asi que mejor cargue el modelo de 125M prompts para que colab no se cuelgue

In [ ]:
gpt = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")
gptQuestions = HappyQuestionAnswering("GPT-NEO", "EleutherAI/gpt-neo-125M")

Some weights of GPTNeoForQuestionAnswering were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Generacion de preguntas
aún se debe de dar la configuracion al gptneo para que no haya repeticion de preguntas.

In [ ]:
generate_question = gptQuestions.answer_question("four plus four equal eight", "how much is 4+4?",top_k=5)
print(generate_question)
print(generate_question[0].answer)

In [ ]:
generate_question = gptQuestions.answer_question("20*100=2000.", "Can you explain how to calculate 20*1000",top_k=1)
print(generate_question)
print(generate_question[0].answer)

[QuestionAnsweringResult(answer='2000', score=0.04837531968951225, start=7, end=11)]
2000


# GENSettings
Acá genero las opciones para agregarle a la consulta
https://happytransformer.com/text-generation/settings/

## generacion texto

In [ ]:
args= GENSettings(no_repeat_ngram_size=2, max_length=100)

In [ ]:
result = gpt.generate_text("Today we are gonna celebrate ", args=args)
print(result.text)

In [ ]:
args_top_k = GENSettings(no_repeat_ngram_size=2, do_sample=True,top_k=50, temperature=0.7)

In [ ]:
result = gpt.generate_text("Today we are gonna celebrate", args=args_top_k)
print(result.text)

## Generacion Pregunta
Intente ver si se podia ingresar argumentos tambien al de pregunta pero no funco.

In [ ]:
args_questions = GENSettings(top_k=90,top_p=3.0, temperature= 0.8, no_repeat_ngram_size=1)

##### Intento de crear una pregunta mas hacia matemáticas

In [ ]:
generate_question = gptQuestions.answer_question("First, we multiply a degree-$1$ term and a degree-$2$ term, so we have a degree-$3$ polynomial. We subtract a constant times a degree-$3$ polynomial, so we can have at most a degree-$3$ polynomial, so at most $4$ terms. However, we are unsure if any of the terms will subtract to zero, so we must multiply out the polynomials:  \\begin{align*}\n&(x+4)(2x^2+3x+9)-3(x^3-2x^2+7x)\\\\\n&\\qquad=x(2x^2+3x+9)+4(2x^2+3x+9)-(3x^3-6x^2+21x)\\\\\n&\\qquad=2x^3+3x^2+9x+8x^2+12x+36-(3x^3-6x^2+21x)\\\\\n&\\qquad=2x^3+11x^2+21x+36-(3x^3-6x^2+21x)\\\\\n&\\qquad=2x^3-3x^3+11x^2+6x^2+21x-21x+36\\\\\n&\\qquad=-x^3+17x^2+36.\n\\end{align*}As we can see, the linear term drops out, and we are left with $\\boxed{3}$ terms.", "How many nonzero terms are in the expansion of $$(x+4)(2x^2+3x+9)-3(x^3-2x^2+7x)?",top_k=2)
print(generate_question)
print(generate_question[0].answer)

# Carga de json
Acá voy a estar cargando el archivo resultado2.json para trabajarlo en un pandas para luego cargarlo.

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    json=fn
mp = pd.read_json(json)


Saving train.json to train (2).json


In [ ]:
mp

,problem,level,type,solution
0,How many vertical asymptotes does the graph of...,Level 3,Algebra,The denominator of the rational function facto...
1,What is the positive difference between $120\%...,Level 1,Algebra,One hundred twenty percent of 30 is $120\cdot3...
2,Find $x$ such that $\lceil x \rceil + x = \dfr...,Level 4,Algebra,"First, we note that $x$ must be positive, sinc..."
3,Evaluate $i^5+i^{-25}+i^{45}$.,Level 5,Algebra,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. ...
4,"If $2^8=4^x$, what is the value of $x$?",Level 1,Algebra,Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Si...
...,...,...,...,...
1182,"When Frederick was born, his grandparents gave...",Level 4,Algebra,"If the interest is simple, then Frederick's mo..."
1183,Simplify the expression $$(x^5+3x^2+3x^5)-(x^7...,Level 4,Algebra,"Combining like terms, we find that \begin{ali..."
1184,"A line goes through point $A(9, 1)$, point $B(...",Level 3,Algebra,The slope of the line is $\frac{1-0}{9-7}=\fra...
1185,If $x = 1 + \frac{x}{1 + \frac{x}{1+ \frac{x}{...,Level 3,Algebra,Notice that the in the equation $$x = 1 + \fra...


In [ ]:
mp_cambiado= mp.rename(columns={"problem":"question","level":"answer_start","type":"context","solution":"answer_text"})
mp_ordenado = mp_cambiado[["context","question","answer_text","answer_start"]]

In [ ]:
mp_ordenado['answer_start'] = mp_ordenado['answer_start'].str.extract(r'(\d+)').astype(int)

In [ ]:
mp_ordenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   context       1187 non-null   object
 1   question      1187 non-null   object
 2   answer_text   1187 non-null   object
 3   answer_start  1187 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 37.2+ KB


In [ ]:
mp_ordenado.to_csv('/content/sample_data/0.csv', index=False, sep=';')

In [ ]:
col_names = ["context","question","answer_text","answer_start"]
test_file = pd.read_csv('/content/sample_data/0.csv',names = col_names , engine="python", sep=';', quotechar='"', error_bad_lines=False)

<ipython-input-84-717503df4024>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test_file = pd.read_csv('/content/sample_data/train.csv',names = col_names , engine="python", sep=';', quotechar='"', error_bad_lines=False)


In [ ]:
test_file.to_csv('/content/sample_data/toychato.csv')

In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
    tsv = fn
test_english = pd.read_csv(tsv,sep=";")

Saving test.csv to test.csv


In [ ]:
col_names = ["context","question","answer_text","answer_start"]
prueba_test_ingles = pd.read_csv('/content/test.csv',names = col_names , engine="python", sep=';', quotechar='"', error_bad_lines=False)

<ipython-input-64-00466c2dcc29>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  prueba_test_ingles = pd.read_csv('/content/test.csv',names = col_names , engine="python", sep=';', quotechar='"', error_bad_lines=False)


In [ ]:
print(f"Train dataset datos: {len(test_english)}")

test_english.info()

Train dataset datos: 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   context       2 non-null      object
 1   question      2 non-null      object
 2   answer_text   2 non-null      object
 3   answer_start  2 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 192.0+ bytes


In [ ]:
Finalprueba

,context,question,answer_text,answer_start
1,Solve the following equation: 2x + 5 = 11,What is the value of x?,"To solve this equation, first, subtract 5 from...",33
2,Factor the following polynomial: x^2 - 4,What are the factors of this polynomial?,"To factor x^2 - 4, we use the difference of sq...",28


In [ ]:
Finalprueba.to_csv('/content/sample_data/prueba_test_ingles.csv')

In [ ]:
from happytransformer import HappyQuestionAnswering, QATrainArgs
# --------------------------------------#
print(f"Train dataset datos: {len(mp_ordenado)}")

# --------------------------------------#
happy_qa = HappyQuestionAnswering()

args = QATrainArgs(num_train_epochs=5)
happy_qa.train('/content/sample_data/prueba_test_ingles.csv')

Train dataset datos: 1187


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Tokenizing data:   0%|          | 0/1 [00:00<?, ? examples/s]

TypeError: ignored

In [ ]:
data = [
    ["Resuelve la siguiente ecuación: 2x + 5 = 11", "¿Cuál es el valor de x?", "Para resolver esta ecuación, primero restamos 5 de ambos lados: 2x = 6. Luego, dividimos ambos lados por 2: x = 3.", 33],
    ["Factoriza el siguiente polinomio: x^2 - 4", "¿Cuáles son los factores de este polinomio?", "Para factorizar x^2 - 4, utilizamos la diferencia de cuadrados: (x + 2)(x - 2).", 28],
    ["Encuentra el valor de la variable y en el sistema de ecuaciones: 3x + 2y = 10 y 2x - y = 4", "¿Cuál es el valor de y?", "Para encontrar el valor de y, puedes resolver el sistema de ecuaciones y encontrar que y = 2.", 37],
    ["Resuelve la siguiente inecuación: 3x + 7 > 22", "¿Cuál es el conjunto de soluciones para x?", "Para resolver la inecuación, primero restamos 7 de ambos lados: 3x > 15. Luego, dividimos ambos lados por 3: x > 5.", 0]
]

column_names = ["context", "question", "answer_text", "answer_start"]

df = pd.DataFrame(data, columns=column_names)
df

,context,question,answer_text,answer_start
0,Resuelve la siguiente ecuación: 2x + 5 = 11,¿Cuál es el valor de x?,"Para resolver esta ecuación, primero restamos ...",33
1,Factoriza el siguiente polinomio: x^2 - 4,¿Cuáles son los factores de este polinomio?,"Para factorizar x^2 - 4, utilizamos la diferen...",28
2,Encuentra el valor de la variable y en el sist...,¿Cuál es el valor de y?,"Para encontrar el valor de y, puedes resolver ...",37
3,Resuelve la siguiente inecuación: 3x + 7 > 22,¿Cuál es el conjunto de soluciones para x?,"Para resolver la inecuación, primero restamos ...",0


############################################################################################

# Hasta el momento me rendi de estar probando con HappyTransformers y comenze a usar las librerias de Pytorch y transformers de HugginFace

In [ ]:
!pip install torch

In [ ]:
from transformers import pipeline

In [ ]:
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')

In [ ]:
prompt = "Today is 10-31-2023 and today is "

In [ ]:
res = generator(prompt, max_length=50, do_sample=True, temperature=0.9)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(res[0]['generated_text'])

Today is 10-31-2023 and today is 
the date of the first meeting of the Board of Directors of 
Cities in the State of Maryland.               


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoTokenizer, GPTNeoForQuestionAnswering
import torch
model = GPTNeoForQuestionAnswering.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

Some weights of GPTNeoForQuestionAnswering were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
question, text = "Who was Jimi hendrix?", "Jimi Hendrix was a groundbreaking and influential American rock guitarist and singer known for his virtuosic playing and innovative approach to music."

In [ ]:
inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index ]
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

predicted_answer = tokenizer.decode(predict_answer_tokens)
print("Respuesta predicha:", predicted_answer)

Respuesta predicha: ?Jimi Hendrix was a groundbreaking and influential American rock guitarist and singer


In [ ]:
print(inputs)

{'input_ids': tensor([[ 8241,   373,  5395,    72,   339,   358,  8609,    30, 18050,    72,
         14666,  8609,   373,   257, 31088,   290, 14212,  1605,  3881, 32705,
           290, 14015,  1900,   329,   465,  4118,    84,   418,   291,  2712,
           290, 13097,  3164,   284,  2647,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
from datasets import load_dataset
import csv
from happytransformer import  HappyQuestionAnswering, QATrainArgs

def main():
    train_csv_path = "/content/sample_data/0.csv"
    eval_csv_path = "/content/sample_data/train.csv"
    train_dataset = load_dataset('squad', split='train[0:499]')
    eval_dataset = load_dataset('squad', split='validation[0:99]')
    generate_csv(train_csv_path, train_dataset)
    generate_csv(eval_csv_path, eval_dataset)

    happy_qa = HappyQuestionAnswering(model_name="EleutherAI/gpt-neo-125M")


    train_args = QATrainArgs(
        num_train_epochs=1,
        learning_rate=1e-5,
        # fp16=True,
        # report_to = ('wandb'),
        # project_name = "happy-transformer-examples",
        # run_name = "question-answering",
        # deepspeed="ZERO-2",

    )
    happy_qa.train(train_csv_path, args=train_args, eval_filepath=eval_csv_path)

    happy_qa.save("finetuned-model/")


def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["context", "question", "answer_text", "answer_start"])
        for case in dataset:
            context = case["context"]
            question = case["question"]
            answer_text = case["answers"]["text"][0]
            answer_start = case["answers"]["answer_start"][0]
            writter.writerow([context, question, answer_text, answer_start])


if __name__ == "__main__":
    main()

Some weights of GPTNeoForQuestionAnswering were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Tokenizing data:   0%|          | 0/499 [00:00<?, ? examples/s]

Tokenizing data:   0%|          | 0/99 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,4.879900,5.082584
50,5.637000,4.717286
100,5.117400,4.536974
150,4.831400,4.382739
200,4.651500,4.267567
250,4.764500,4.103709
300,4.328400,4.031367
350,4.727800,4.023115
400,4.343200,3.992317
450,4.338300,3.996045


In [ ]:
happy_qa = HappyQuestionAnswering()
result = happy_qa.answer_question("Algebra", "How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?")
print(result)

[QuestionAnsweringResult(answer='Algebra', score=0.7261653542518616, start=0, end=7)]
